In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import os 
from datetime import timedelta, date
import geopandas as gpd 

In [2]:
#df_ais_entering_port = pd.read_csv('./assets/test/ais_entering_port.csv')
df_ais_entering_port = pd.read_csv('./data/ais_with_vessel_berth/ais_entering_port.csv')

In [3]:
df_ais_entering_port.shape

(3381018, 20)

In [4]:
# create geopandas dataframe
gdf_vessel = gpd.GeoDataFrame(df_ais_entering_port, geometry=gpd.points_from_xy(df_ais_entering_port['lon'], df_ais_entering_port['lat']))

In [5]:
geojson_dir = './data/geojson/'
geojson_file_list= []
for file in os.listdir(geojson_dir):
    if file.endswith('geojson'):
        geojson_file_list.append(file)

print(geojson_file_list)

['PierA.geojson', 'PierC.geojson', 'PierE.geojson', 'PierG.geojson', 'PierJ.geojson', 'PierT.geojson']


In [6]:
#geojson_file_list = geojson_file_list[:1]

In [7]:
#geojson_file_list

In [8]:
# create columns 'at_terminal' and 'terminal_name'
gdf_vessel.loc[:, 'at_terminal'] = False
gdf_vessel.loc[:, 'terminal_name'] = np.nan

for file in geojson_file_list:
    # read geojson file for LBCT
    polygon = gpd.read_file(os.path.join(geojson_dir, file))
    terminal_name = file.split('.')[0]
    # geofence for the 6 piers in Port of LB. Set 'at_terminal' to True and add name to 'terminal_name' when it is wihtin the polygon 
    mask = (polygon.loc[0, 'geometry'])
    pip_mask_geofence = gdf_vessel.within(mask)
    print(f"Terminal: {terminal_name}, Value Count: {pip_mask_geofence.value_counts()}") 
    gdf_vessel.loc[pip_mask_geofence, 'at_terminal'] = pip_mask_geofence
    gdf_vessel.loc[pip_mask_geofence, 'terminal_name'] = terminal_name

Terminal: PierA, Value Count: False    3315695
True       65323
dtype: int64
Terminal: PierC, Value Count: False    3363142
True       17876
dtype: int64
Terminal: PierE, Value Count: False    3328385
True       52633
dtype: int64
Terminal: PierG, Value Count: False    3365004
True       16014
dtype: int64
Terminal: PierJ, Value Count: False    3341568
True       39450
dtype: int64
Terminal: PierT, Value Count: False    3296413
True       84605
dtype: int64


In [9]:
gdf_vessel.columns.values

array(['mmsi', 'time_seen', 'lat', 'lon', 'sog', 'cog', 'heading',
       'vessel_name', 'imo', 'call_sign', 'vessel_type', 'status',
       'length', 'width', 'draft', 'cargo', 'transceiver_class',
       'lat_lon', 'date_seen', 'is_entering_port', 'geometry',
       'at_terminal', 'terminal_name'], dtype=object)

In [10]:
gdf_vessel['terminal_name'].value_counts()

PierT    84605
PierA    65323
PierE    52633
PierJ    39450
PierC    17876
PierG    16014
Name: terminal_name, dtype: int64

In [11]:
gdf_vessel['at_terminal'].value_counts()

False    3105117
True      275901
Name: at_terminal, dtype: int64

In [12]:
gdf_vessel.head(3)

,mmsi,time_seen,lat,lon,sog,cog,heading,vessel_name,imo,call_sign,...,width,draft,cargo,transceiver_class,lat_lon,date_seen,is_entering_port,geometry,at_terminal,terminal_name
0,565807000.0,2020-01-01 00:00:00,32.31175,-117.53433,14.7,162.1,163.0,NYK CLARA,IMO9355408,9VFW9,...,30.0,11.5,72.0,A,"32.3118,-117.5343",2020-01-01,False,POINT (-117.53433 32.31175),False,NaN
1,565807000.0,2020-01-01 00:31:18,32.18897,-117.48776,14.7,160.0,158.0,NYK CLARA,IMO9355408,9VFW9,...,30.0,11.5,72.0,A,"32.189,-117.4878",NaN,False,POINT (-117.48776 32.18897),False,NaN
2,565807000.0,2020-01-01 01:00:00,32.08009,-117.43979,14.5,161.7,163.0,NYK CLARA,IMO9355408,9VFW9,...,30.0,11.5,72.0,A,"32.0801,-117.4398",NaN,False,POINT (-117.43979 32.08009),False,NaN


In [13]:
gdf_vessel.columns.values

array(['mmsi', 'time_seen', 'lat', 'lon', 'sog', 'cog', 'heading',
       'vessel_name', 'imo', 'call_sign', 'vessel_type', 'status',
       'length', 'width', 'draft', 'cargo', 'transceiver_class',
       'lat_lon', 'date_seen', 'is_entering_port', 'geometry',
       'at_terminal', 'terminal_name'], dtype=object)

In [14]:
# pick required columns 
df_vessel_lbct = gdf_vessel[
    ['mmsi',
    'time_seen',
    'lat',
    'lon',
    'sog',
    'cog',
    'heading',
    'vessel_name',
    'imo',
    'call_sign',
    'vessel_type',
    'status',
    'length',
    'width',
    'draft',
    'cargo',
    'transceiver_class',
    'date_seen',
    'is_entering_port',
    'at_terminal',
    'terminal_name']
]

In [15]:
#df_vessel_lbct.to_csv('./assets/test/geofenced_ais.csv', index=False)

df_vessel_lbct.to_csv('./data/ais_with_vessel_berth/geofenced_ais.csv', index=False)

In [16]:
df_ais_entering_port.shape

(3381018, 23)

In [17]:
df_vessel_lbct.shape

(3381018, 21)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6b18b33d-3a56-4f49-ad6e-71ecea9f0183' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>